<a href="https://colab.research.google.com/github/kaifkh20/d2l/blob/main/linear_reg_fs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
# !pip install d2l
%matplotlib inline
import torch
from d2l import torch as d2l


In [35]:
class LinearRegressionScratch(d2l.Module):
    """The linear regression model implemented from scratch."""
    def __init__(self, num_inputs, lr, sigma=0.01):
        super().__init__()
        self.save_hyperparameters()
        self.w = torch.normal(0, sigma, (num_inputs, 1), requires_grad=True)
        self.b = torch.zeros(1, requires_grad=True)


In [27]:
@d2l.add_to_class(LinearRegressionScratch)
def forward(self,X):
  return torch.matmul(X,self.w)+self.b

In [28]:
@d2l.add_to_class(LinearRegressionScratch)
def loss(self, y_hat, y):
    l = (y_hat - y) ** 2 / 2
    return l.mean()

In [39]:
class SGD(d2l.HyperParameters):
    """Minibatch stochastic gradient descent."""
    def __init__(self, params, lr):
        self.save_hyperparameters()

    def step(self):
        for param in self.params:
            param -= self.lr * param.grad
    """Imagine you're walking down a hill (minimizing loss) by taking small steps based
      on the slope (gradient). zero_grad is like resetting your compass (gradient)
      at the beginning of each step, ensuring you're basing your next move solely
      on the current terrain, not the accumulated path you've already taken.
    """
    def zero_grad(self):
        for param in self.params:
            if param.grad is not None:
                param.grad.zero_()

@d2l.add_to_class(LinearRegressionScratch)
def configure_optimizers(self):
    return SGD([self.w, self.b], self.learning_rate)


In [30]:
# In each epoch, we iterate through the entire training dataset, passing once through every example
# (assuming that the number of examples is divisible by the batch size).
# In each iteration, we grab a minibatch
# of training examples, and compute its loss through the model’s training_step method.

In [31]:
@d2l.add_to_class(d2l.Trainer)
def prepare_batch(self, batch):
    return batch

@d2l.add_to_class(d2l.Trainer)
def fit_epoch(self):
    self.model.train()
    for batch in self.train_dataloader:
        loss = self.model.training_step(self.prepare_batch(batch))
        self.optim.zero_grad()
        with torch.no_grad():
            loss.backward() # calculate gradients for every param that requires it.
            if self.gradient_clip_val > 0:  # To be discussed later
                self.clip_gradients(self.gradient_clip_val, self.model)
            self.optim.step()
        self.train_batch_idx += 1
    if self.val_dataloader is None:
        return
    self.model.eval()
    for batch in self.val_dataloader:
        with torch.no_grad():
            self.model.validation_step(self.prepare_batch(batch))
        self.val_batch_idx += 1


In [42]:
model = LinearRegressionScratch(2,lr=0.03)
data = d2l.SyntheticRegressionData(w=torch.tensor([2, -3.4]), b=4.2)
trainer = d2l.Trainer(max_epochs=3)
trainer.fit(model, data)

AttributeError: 'LinearRegressionScratch' object has no attribute 'learning_rate'